In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio.features

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer as pc
import xrspatial.multispectral as ms
import odc
from odc.stac import stac_load

# Pass your API key here
pc.settings.set_subscription_key('1b6b6eb7c08a4e58a88ab5e9062492f3')

from tqdm import tqdm

In [2]:
crop_presence_data = pd.read_csv("Crop_Location_Data_20221201_1_to_201.csv")
crop_presence_data

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice
...,...,...
195,"(9.996682289726301, 105.5005520032351)",Rice
196,"(9.99259423025936, 105.52008384290963)",Rice
197,"(9.996228060896641, 105.51463309695396)",Rice
198,"(9.989868857281401, 105.52053807173928)",Rice


In [3]:
RVI_values=[]
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    lat_long=coordinates.replace('(','').replace(')','').replace(' ','').split(',')
    #box_size_deg = 0
    min_lon = float(lat_long[1])
    min_lat = float(lat_long[0])
    max_lon = float(lat_long[1])
    max_lat = float(lat_long[0])
    bounds = (min_lon, min_lat, max_lon, max_lat)
    time_window="2021-12-01/2022-01-31"
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bounds, datetime=time_window)
    items = list(search.get_all_items()) # This produces a list of scene IDs
    resolution = 20  # meters per pixel 
    scale = resolution / 111320.0 # degrees per pixel for CRS:4326 
    data = stac_load(items,bands=["vv", "vh"], patch_url=pc.sign, bbox=bounds, crs="EPSG:4326", resolution=scale)
    mean = data.mean(dim=['latitude','longitude']).compute()
    dop = (mean.vv / (mean.vv + mean.vh))
    m = 1-dop
    rvi = (np.sqrt(dop))*((4*mean.vh)/(mean.vv + mean.vh))
    
    np_array_rvi=np.array(rvi.values)
    average_rvi=np.mean(np_array_rvi)
    RVI_values.append(average_rvi)
    #RVI_values.append(np_array_rvi)
    

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [1:24:29<00:00, 25.35s/it]


In [4]:
RVI_values

[0.72877026,
 0.52279985,
 0.5133395,
 0.4821024,
 0.5398251,
 0.48709226,
 0.6375164,
 0.5986355,
 0.72174144,
 0.50928473,
 0.6810851,
 0.51703584,
 0.5939318,
 0.65432435,
 0.6090058,
 0.57580745,
 0.4559156,
 0.64110607,
 0.5216725,
 0.6185254,
 0.44271794,
 0.6753324,
 0.56371224,
 0.36877707,
 0.49350205,
 0.51083624,
 0.53528947,
 0.56233203,
 0.34927982,
 0.46476036,
 0.760416,
 0.661716,
 0.5589565,
 0.59003925,
 0.65633315,
 0.3905905,
 0.56556207,
 0.34124875,
 0.4450949,
 0.60655373,
 0.50741166,
 0.60652095,
 0.53346175,
 0.52655303,
 0.45423698,
 0.5811678,
 0.519626,
 0.35575628,
 0.67449635,
 0.50887156,
 0.60261935,
 0.5096119,
 0.37405777,
 0.6991469,
 0.6039809,
 0.57951206,
 0.4561821,
 0.53812087,
 0.57488865,
 0.56338257,
 0.3656815,
 0.509909,
 0.4236485,
 0.48005956,
 0.5231621,
 0.63336354,
 0.61638594,
 0.41018534,
 0.46978888,
 0.6851476,
 0.5208756,
 0.41882315,
 0.4779707,
 0.7046133,
 0.6795983,
 0.5152009,
 0.44087282,
 0.6449467,
 0.55618703,
 0.5814037,

In [5]:
crop_presence_data['RVI'] = RVI_values

In [6]:
crop_presence_data

,Latitude and Longitude,Class of Land,RVI
0,"(10.323727047081501, 105.2516346045924)",Rice,0.728770
1,"(10.322364360592521, 105.27843410554115)",Rice,0.522800
2,"(10.321455902933202, 105.25254306225168)",Rice,0.513340
3,"(10.324181275911162, 105.25118037576274)",Rice,0.482102
4,"(10.324635504740822, 105.27389181724476)",Rice,0.539825
...,...,...,...
195,"(9.996682289726301, 105.5005520032351)",Rice,0.809396
196,"(9.99259423025936, 105.52008384290963)",Rice,0.758639
197,"(9.996228060896641, 105.51463309695396)",Rice,0.738626
198,"(9.989868857281401, 105.52053807173928)",Rice,0.780588


In [7]:
crop_presence_data.to_csv('lat_lon1_1to201_Rvi.csv', index=False)